##### LLM Document Analysis
##### Louise Barros
##### 03/26/2025

In [1]:
import pandas as pd
import requests
import ollama
import json
import time
from bs4 import BeautifulSoup


req_headers = {"User-Agent": "lo588183@ucf.edu"} 

def find_cik_for_ticker(symbol, headers=req_headers):
    """
    Determines the 10-digit CIK (Central Index Key) for a given stock symbol.
    Fetches the SEC's master JSON at:
    https://www.sec.gov/files/company_tickers.json
    
    Args:
        symbol (str): The company's ticker (e.g., 'AAPL').
        headers (dict): HTTP headers for the GET request.

    Returns:
        str: A zero-padded 10-digit string representing the CIK.

    Raises:
        ValueError: If the ticker is not found in the SEC JSON data.
    """
    # Normalize ticker by replacing '.' with '-'
    formatted_symbol = symbol.upper().replace(".", "-")
    resp = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
    resp.raise_for_status()
    all_tickers = resp.json()

    for entry in all_tickers.values():
        if entry["ticker"] == formatted_symbol:
            return str(entry["cik_str"]).zfill(10)
    
    raise ValueError(f"Symbol {symbol} not found in the SEC database.")


def pull_submission_data_for_symbol(symbol, headers=req_headers, only_filings=False):
    """
    Retrieves either the full JSON "submissions" data or just the recent filings DataFrame
    for the specified stock symbol from the SEC "submissions" endpoint.

    Args:
        symbol (str): The company's ticker (e.g. 'WSM').
        headers (dict): HTTP headers for requests.
        only_filings (bool): If True, returns only the 'filings.recent' data as a DataFrame.

    Returns:
        dict or pd.DataFrame: The full submission JSON or a DataFrame of the recent filings.
    """
    cik_id = find_cik_for_ticker(symbol, headers=headers)
    url = f"https://data.sec.gov/submissions/CIK{cik_id}.json"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    json_data = resp.json()

    if only_filings:
        return pd.DataFrame(json_data["filings"]["recent"])
    else:
        return json_data

def extract_8k_forms(symbol, just_accessions=False, headers=req_headers):
    """
    Retrieves a DataFrame of recent 8-K forms for a specific stock ticker.
    If just_accessions=True, returns only the accession numbers Series.

    Args:
        symbol (str): The company's ticker symbol (e.g. 'WSM').
        just_accessions (bool): Whether to return only the accession numbers.
        headers (dict): Headers to use for the requests.

    Returns:
        pd.DataFrame or pd.Series: 8-K filings data or just the accession numbers.
    """
    filings_df = pull_submission_data_for_symbol(symbol, headers=headers, only_filings=True)
    eights_df = filings_df[filings_df["form"] == "8-K"]

    if just_accessions:
        eights_df = eights_df.set_index("reportDate")
        return eights_df["accessionNumber"]
    else:
        return eights_df

# A dictionary mapping certain tickers to their official company names
COMPANY_NAME_MAP = {
    "SHOP": "Shopify Inc.",                        
    "ABT": "Abbott Laboratories",                  
    "NOW": "ServiceNow, Inc.",   
    "TTD": "The Trade Desk, Inc.",                
    "ROKU": "Roku, Inc.",
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "TSLA": "Tesla, Inc.",
    "AMZN": "Amazon.com, Inc.",
    "NVDA": "NVIDIA Corporation",
    "META": "Meta Platforms, Inc.",
    "INTC": "Intel Corporation",
    "AMD": "Advanced Micro Devices, Inc.",
    "NFLX": "Netflix, Inc.",
    "CRM": "Salesforce, Inc.",
    "ADBE": "Adobe Inc.",
    "PYPL": "PayPal Holdings, Inc.",
    "UBER": "Uber Technologies, Inc.",
    "SBUX": "Starbucks Corporation",
    "PFE": "Pfizer Inc.",
    "MRK": "Merck & Co., Inc.",
    "JNJ": "Johnson & Johnson"
}

# Set of keywords that we want to ignore if they appear in product descriptions
BLOCKED_FINANCIAL_TERMS = {
    "notes", "bond", "issuance", "due",
    "fiscal", "earning", "quarter", "result"
}

def product_passes_filter(product_name: str, product_desc: str) -> bool:
    """
    Simple check to avoid purely financial references based on a set of keywords.
    
    Returns False if product_name + product_desc mention terms like 'bond', 'quarter', etc.
    """
    combo_text = (product_name + " " + product_desc).lower()
    return not any(fin_term in combo_text for fin_term in BLOCKED_FINANCIAL_TERMS)

def fix_description_capitalization(description: str) -> str:
    """
    Ensures the product description starts with a capital letter if it's alphabetical.
    """
    clean_desc = description.strip()
    if clean_desc and clean_desc[0].isalpha():
        clean_desc = clean_desc[0].upper() + clean_desc[1:]
    return clean_desc

def get_8k_product_details_via_llm(symbol, headers=req_headers):
    """
    1) Pull the last ~10 8-K filings for the given symbol.
    2) For each filing, fetch its primary document from EDGAR, parse the text,
       then call an LLM (ollama) to extract product mentions belonging to the symbol's company.
    3) Filter out purely financial references, fix capitalization.
    4) Return a DataFrame with columns:
       [Company Name, Stock Name, Filing Time, New Product, Product Description].
    """
    # Grab only the 8-K forms
    forms_df = extract_8k_forms(symbol, headers=headers)
    if forms_df.empty:
        return pd.DataFrame()

    # Sort by date descending, only keep top 10
    forms_df = forms_df.sort_values("filingDate", ascending=False).head(10)

    # Look up the company's formal name from the map, fallback to the symbol
    official_name = COMPANY_NAME_MAP.get(symbol, symbol)

    compiled_data = []

    for _, row in forms_df.iterrows():
        # Build the link to the filing
        filing_link = (
            f"https://www.sec.gov/Archives/edgar/data/"
            f"{find_cik_for_ticker(symbol)}/"
            f"{row['accessionNumber'].replace('-', '')}/"
            f"{row['primaryDocument']}"
        )

        # Fetch the filing
        response = requests.get(filing_link, headers=headers)
        if response.status_code != 200:
            print(f"❌ Could not fetch filing: {filing_link}")
            continue

        # Extract text from HTML
        text_content = BeautifulSoup(response.text, "html.parser").get_text(" ", strip=True)
        truncated_text = text_content[:4000]  # Truncate for the LLM context limit

        # Prepare the LLM prompt
        prompt_text = f"""
You are analyzing an SEC 8-K filing for {official_name}.
Extract ONLY new products belonging to {official_name}.
Ignore references to other brand names or companies (for example, if you see Tesla but your company is Apple, skip it).

Also ignore:
- financial results, earnings, or quarters
- stock splits or executive changes
- acquisitions, intangible assets, or purely financial data

Return strictly JSON, like:
{{
  "products": [
    {{
      "product_name": "string",
      "product_description": "string"
    }}
  ]
}}
No disclaimers. If no product for {official_name}, return {{ "products": [] }}.

If no product for {official_name}, return {{ "products": [] }}. No extra text or disclaimers.

Filing text (truncated):
{truncated_text}
"""

        # Invoke the LLM via ollama.chat
        llm_response = ollama.chat(
            model="llama3.2",
            messages=[{"role": "user", "content": prompt_text}]
        )
        llm_output = llm_response["message"]["content"].strip()

        print(f"\n🔹LLM result from {filing_link}:\n{llm_output}\n")

        # Parse the JSON
        try:
            parsed_json = json.loads(llm_output)
            if isinstance(parsed_json, dict) and "products" in parsed_json:
                products_list = parsed_json["products"]
                if isinstance(products_list, list):
                    for product_info in products_list:
                        p_name = product_info.get("product_name", "").strip()
                        p_desc = product_info.get("product_description", "").strip()
                        p_desc = p_desc[:180]  # Limit to 180 chars for consistency

                        if p_name and product_passes_filter(p_name, p_desc):
                            p_desc = fix_description_capitalization(p_desc)
                            compiled_data.append({
                                "Company Name": official_name,
                                "Stock Name": symbol,
                                "Filing Time": row["filingDate"],
                                "New Product": p_name,
                                "Product Description": p_desc
                            })
                else:
                    print(f"❌ 'products' is not a list in LLM JSON for {filing_link}")
            else:
                print(f"❌ 'products' key missing in LLM JSON for {filing_link}")
        except json.JSONDecodeError:
            print(f"❌ Could not parse JSON from LLM for {filing_link}:\n{llm_output}\n")

    return pd.DataFrame(compiled_data)

if __name__ == "__main__":
    # Example set of tickers to process
    desired_tickers = [
        "SHOP", "ABT", "NOW", "TTD", "ROKU", "AAPL", "MSFT", "TSLA", "AMZN", "NVDA", "META", 
        "INTC", "AMD", "NFLX", "CRM", "ADBE", "PYPL", "UBER", "SBUX", "PFE","MRK", "JNJ"
    ]

    # We accumulate data from all these tickers
    result_frames = []

    for ticker_sym in desired_tickers:
        print(f"\n=== Extracting 8-K product info for {ticker_sym} ===\n")
        output_df = get_8k_product_details_via_llm(ticker_sym, headers=req_headers)
        result_frames.append(output_df)
        time.sleep(1)  # Slight delay to be polite to SEC's servers

    # Combine all results
    final_results = pd.concat(result_frames, ignore_index=True)

    # Remove any rows where the product description is missing or trivial
    final_results.dropna(subset=["Product Description"], inplace=True)
    final_results = final_results[final_results["Product Description"].str.strip() != ""]
    final_results = final_results[
        ~final_results["Product Description"].str.lower().isin([
            "string", "none", "null", "no product description provided",
            "not available in filing text"
        ])
    ]

    # Save to a CSV file
    final_results.to_csv("8k_extractions_louise.csv", index=False)
    print("\nProcess complete! Data stored in 8k_extractions_louise.csv.")



=== Extracting 8-K product info for SHOP ===


🔹LLM result from https://www.sec.gov/Archives/edgar/data/0001594805/000159480525000018/shop-20250318.htm:
{
  "products": [
    {
      "product_name": "Shopify Plus",
      "product_description": "Enterprise plan for Shopify"
    },
    {
      "product_name": "Shopify Payments",
      "product_description": "Payment processing service"
    },
    {
      "product_name": "Shopify Shipping",
      "product_description": "Shipping service"
    }
  ]
}


🔹LLM result from https://www.sec.gov/Archives/edgar/data/0001594805/000159480525000011/shop-20250211.htm:
{
  "products": [
    {
      "product_name": "Shopify Payments Advanced",
      "product_description": "Advanced payment processing capabilities for Shopify merchants"
    },
    {
      "product_name": "Shopify Payments Basic",
      "product_description": "Basic payment processing capabilities for Shopify merchants"
    }
  ]
}


=== Extracting 8-K product info for ABT ===


🔹LLM res